<a href="https://colab.research.google.com/github/SPtestZ7Z/Dash_occupation_data/blob/main/Labour_market_visualisation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LABOUR MARKET DASH APP**
v2: Trends

# **Set up**

**Import packages**

In [23]:
import pandas as pd

In [24]:
!pip install dash jupyter-dash pandas plotly
import dash
from dash import dcc, html, Input, Output, callback
import plotly.express as px
import pandas as pd
import re

Mount and access google drive files

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load data**

Load data (originally from https://www.nomisweb.co.uk/sources/aps)

In [26]:
path2010 = "https://docs.google.com/spreadsheets/d/1XSHmlzIctVZHB4bJn3sedgtBSTpmNeL2tNvI3xnTD9g/edit?usp=sharing"
path2015 = "https://docs.google.com/spreadsheets/d/1eSiKyP1M0bDoKM03RG89S0l1o3TtSQg618LxKTSIhV4/edit?usp=sharing"
path2021 = "https://docs.google.com/spreadsheets/d/1--Bvrfit7ZGZgNycP0_-fDAUIEf28OkwsFdyhsuJCKs/edit?usp=sharing"

Read data from googlesheets

In [27]:
!pip install --upgrade gspread google-auth

import pandas as pd
from google.colab import auth
import gspread
from google.auth import default

# Authenticate with Google Colab
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet by name or URL
# Replace 'your_sheet_name_or_url' with your sheet name or URL
# The error was caused because path1, path2, path3 were not valid Google Sheet URLs.
# They should be in the format: 'https://docs.google.com/spreadsheets/d/<sheet_id>/edit?usp=sharing'
# where <sheet_id> is the unique identifier of your sheet.

sheet1 = gc.open_by_url(path2010).sheet1
sheet2 = gc.open_by_url(path2015).sheet1
sheet3 = gc.open_by_url(path2021).sheet1


# Convert the sheet data to a Pandas DataFrame
df2010 = pd.DataFrame(sheet1.get_all_records())
df2015 = pd.DataFrame(sheet2.get_all_records())
df2021 = pd.DataFrame(sheet3.get_all_records())
df2010.head(5)

,Occupation,country:United Kingdom,Confcountry:United Kingdom,country:Great Britain,Confcountry:Great Britain,country:England,Confcountry:England,country:Wales,Confcountry:Wales,country:Scotland,...,gor:South East,Confgor:South East,gor:South West,Confgor:South West,gor:Wales,Confgor:Wales,gor:Scotland,Confgor:Scotland,gor:Northern Ireland,Confgor:Northern Ireland
0,Total,28929900,149900,28159900,147400,24378700,147800,1312800,21300,2468400,...,4176200,65200,2499000,46600,1312800,21300,2468400,33400,770000,28400
1,"1 Managers, directors and senior officials",2922000,47700,2860500,47000,2526100,47600,119300,6400,215100,...,477400,22000,275700,15500,119300,6400,215100,9800,61500,8000
2,11 Corporate managers and directors,2088400,40300,2048600,39800,1819100,40400,82000,5300,147500,...,351000,18900,184200,12600,82000,5300,147500,8200,39900,6500
3,111 Chief Executives and Senior Officials,71100,7400,70600,7400,60600,7400,3900,1200,6100,...,12300,3500,5100,2100,3900,1200,6100,1700,600,!
4,1115 Chief executives and senior officials,60300,6800,59700,6800,51800,6800,3200,1000,4800,...,10800,3300,4300,1900,3200,1000,4800,1500,600,!


In [28]:
#Check data frames are OK
print(f"df1=\n{df2021.head()}")
#df2015.dtypes


df1=
                                   Occupation country:United Kingdom  \
0                                       Total               32130300   
1  1 Managers, directors and senior officials                3660900   
2         11 Corporate managers and directors                2472400   
3   111 Chief Executives and Senior Officials                 157300   
4  1115 Chief executives and senior officials                 144600   

  Confcountry:United Kingdom country:Great Britain Confcountry:Great Britain  \
0                     205000              31283400                    203900   
1                      69200               3588200                     69000   
2                      56900               2425700                     56800   
3                      14300                155100                     14400   
4                      13700                142400                     13800   

  country:England Confcountry:England country:Wales Confcountry:Wales  \
0       

In [29]:
#Convert values to floats
import pandas as pd
for col in df2010.columns:
    if col != 'Occupation':
        df2010[col] = pd.to_numeric(df2010[col], errors='coerce')
    else:
        print("occupation 2010")

for col in df2015.columns:
    if col != 'Occupation':
        df2015[col] = pd.to_numeric(df2015[col], errors='coerce')
    else:
        print("occupation 2015")


for col in df2021.columns:
    if col != 'Occupation':
        df2021[col] = pd.to_numeric(df2021[col], errors='coerce')
    else:
        print("occupation 2021")


occupation 2010
occupation 2015
occupation 2021


In [30]:
df2010.head()

,Occupation,country:United Kingdom,Confcountry:United Kingdom,country:Great Britain,Confcountry:Great Britain,country:England,Confcountry:England,country:Wales,Confcountry:Wales,country:Scotland,...,gor:South East,Confgor:South East,gor:South West,Confgor:South West,gor:Wales,Confgor:Wales,gor:Scotland,Confgor:Scotland,gor:Northern Ireland,Confgor:Northern Ireland
0,Total,28929900,149900.0,28159900,147400.0,24378700,147800.0,1312800.0,21300.0,2468400.0,...,4176200.0,65200.0,2499000.0,46600.0,1312800.0,21300.0,2468400.0,33400.0,770000.0,28400.0
1,"1 Managers, directors and senior officials",2922000,47700.0,2860500,47000.0,2526100,47600.0,119300.0,6400.0,215100.0,...,477400.0,22000.0,275700.0,15500.0,119300.0,6400.0,215100.0,9800.0,61500.0,8000.0
2,11 Corporate managers and directors,2088400,40300.0,2048600,39800.0,1819100,40400.0,82000.0,5300.0,147500.0,...,351000.0,18900.0,184200.0,12600.0,82000.0,5300.0,147500.0,8200.0,39900.0,6500.0
3,111 Chief Executives and Senior Officials,71100,7400.0,70600,7400.0,60600,7400.0,3900.0,1200.0,6100.0,...,12300.0,3500.0,5100.0,2100.0,3900.0,1200.0,6100.0,1700.0,600.0,NaN
4,1115 Chief executives and senior officials,60300,6800.0,59700,6800.0,51800,6800.0,3200.0,1000.0,4800.0,...,10800.0,3300.0,4300.0,1900.0,3200.0,1000.0,4800.0,1500.0,600.0,NaN


In [31]:
df2010['Occupation'] = df2010['Occupation'].astype(str)
df2015['Occupation'] = df2015['Occupation'].astype(str)
df2021['Occupation'] = df2021['Occupation'].astype(str)

# **Dash App II**

**App II**

Additions: these key features:

**Occupation Selection:**

Added a multi-select dropdown that lets users pick up to 5 occupations from the complete list across all years
The dropdown is populated with all unique occupation names from df2010, df2015, and df2021
Added a warning message if a user tries to select more than 5 occupations

**Trend Line Chart:**

Created a line chart showing how selected occupations change across 2010, 2015, and 2021. Each occupation gets its own line with distinct color and markers
The y-axis shows the number of people in the "country:United Kingdom" column
Included proper formatting for numbers with commas

**Dashboard Layout:**

Organized the dashboard with clear section headers
Added spacing between components for better visual hierarchy
Kept all your original bar chart functionality intact
Made the overall dashboard more cohesive with consistent styling

**User Experience Improvements:**

Added instructions for users
Shows a placeholder message when no occupations are selected
Shortened very long occupation names in the legend for better readability
Included hover functionality to see exact values

The implementation uses Plotly's Graph Objects for more control over the line chart styling, while keeping all the improvements we made to the bar chart. The trend chart allows users to easily compare how different occupations have changed over the three time periods.

**Adition of second chart**

In [36]:
import dash
from dash import dcc, html, Input, Output, callback
import plotly.express as px
import plotly.graph_objects as go
import re
import pandas as pd
import textwrap

# Define a function to extract Tier 1 categories
def extract_tier1(df):
    tier1_data = {}
    for index, row in df.iterrows():
        occupation = str(row['Occupation'])
        match = re.match(r"^[1-9]\s", occupation)  # Match only single digit 1-9 followed by a space
        if match:
            tier1_category = occupation  # Store the full occupation title
            if tier1_category not in tier1_data:
                tier1_data[tier1_category] = 0
            tier1_data[tier1_category] += row['country:United Kingdom']
    print(tier1_data)
    return tier1_data

# Function to get all unique occupations from all dataframes
def get_all_occupations():
    all_occupations = set()
    for df in [df2010, df2015, df2021]:
        occupations = df['Occupation'].unique().tolist()
        all_occupations.update(occupations)
    return sorted(list(all_occupations))

# Create Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("UK Occupation Analysis Dashboard", style={'textAlign': 'center', 'marginBottom': '30px'}),

    # First section - Bar chart
    html.Div([
        html.H2("Occupation Distribution by Tier 1"),
        dcc.Dropdown(
            id="year-dropdown",
            options=[
                {"label": "2010", "value": 2010},
                {"label": "2015", "value": 2015},
                {"label": "2021", "value": 2021},
            ],
            value=2010
        ),
        dcc.Graph(id="occupation-bar-chart", style={'height': '700px'})
    ], style={'marginBottom': '50px'}),

    # Second section - Trend chart
    html.Div([
        html.H2("Occupation Trends Over Time (2010-2021)"),
        html.P("Select up to 10 occupations to compare their trends:"),
        dcc.Dropdown(
            id="occupation-dropdown",
            options=[{"label": occupation, "value": occupation} for occupation in get_all_occupations()],
            multi=True,
            value=[],  # Default empty
            placeholder="Select up to 10 occupations..."
        ),
        html.Div(id="selection-warning", style={'color': 'red', 'marginTop': '10px'}),
        dcc.Graph(id="occupation-trend-chart", style={'height': '600px'})
    ])
])


# Define a callback to update the bar chart based on the selected year
@callback(
    Output("occupation-bar-chart", "figure"),
    Input("year-dropdown", "value")
)
def update_bar_chart(selected_year):
    # Choose data
    if selected_year == 2010:
        tier1_data = extract_tier1(df2010)
        df = pd.DataFrame({'Occupation': tier1_data.keys(), 'Count': tier1_data.values()})
    elif selected_year == 2015:
        tier1_data = extract_tier1(df2015)
        df = pd.DataFrame({'Occupation': tier1_data.keys(), 'Count': tier1_data.values()})
    elif selected_year == 2021:
        tier1_data = extract_tier1(df2021)
        df = pd.DataFrame({'Occupation': tier1_data.keys(), 'Count': tier1_data.values()})
    else:
        return {}  # Handle invalid input

    # Clean occupation labels - extract the main part after the number
    df['Occupation_Clean'] = df['Occupation'].apply(lambda x: re.sub(r'^[0-9]\s+', '', x))

    # Create the base figure
    fig = px.bar(df, x='Occupation_Clean', y='Count',
                 labels={'Occupation_Clean': 'Tier 1 Occupation', 'Count': 'Number of People'})

    # Prepare wrapped labels - limit to 15 characters per line for better readability
    wrapped_labels = []
    for text in df['Occupation_Clean']:
        # Split long text into multiple lines with a maximum of 15 characters per line
        wrapped_text = '<br>'.join(textwrap.wrap(text, width=15))
        wrapped_labels.append(wrapped_text)

    # Configure axes and layout for better visualization
    fig.update_layout(
        # Increase chart height
        height=650,
        margin=dict(l=50, r=50, t=50, b=200),  # Increase bottom margin for labels

        # Configure y-axis with 1 million increments
        yaxis=dict(
            title='Number of People',
            tickformat=',d',  # Format numbers with commas
            dtick=1000000,    # Set tick interval to 1 million
            title_standoff=25  # Add some space between axis and title
        ),

        # Configure x-axis with better spacing
        xaxis=dict(
            title='Tier 1 Occupation',
            tickangle=0,      # Keep labels horizontal
            title_standoff=25
        ),

        # Improve overall spacing
        bargap=0.3           # Increase gap between bars for better label spacing
    )

    # Apply the wrapped labels to x-axis with larger font
    fig.update_xaxes(
        tickfont=dict(size=12),  # Increased font size
        ticktext=wrapped_labels,  # Use our wrapped labels
        tickvals=df['Occupation_Clean']  # Position labels correctly
    )

    return fig


# Define callbacks for the trend chart
@callback(
    [Output("occupation-trend-chart", "figure"),
     Output("selection-warning", "children")],
    [Input("occupation-dropdown", "value")]
)
def update_trend_chart(selected_occupations):
    # Check if too many occupations are selected
    warning_message = ""
    if len(selected_occupations) > 10:
        selected_occupations = selected_occupations[:10]  # Take only the first 5
        warning_message = "Maximum 10 occupations can be displayed. Only showing the first 5 selected."

    # Create a new dataframe to store the trend data
    trend_data = []

    # Get data for each year
    for year, df in [(2010, df2010), (2015, df2015), (2021, df2021)]:
        for occupation in selected_occupations:
            # Find the occupation in the dataframe
            row = df[df['Occupation'] == occupation]
            if not row.empty:
                value = row['country:United Kingdom'].values[0]
                trend_data.append({
                    'Year': year,
                    'Occupation': occupation,
                    'Value': value
                })

    # Convert to dataframe
    trend_df = pd.DataFrame(trend_data)

    # Create the line chart using plotly.graph_objects for more control
    fig = go.Figure()

    # If no occupations selected, return empty chart with a message
    if not selected_occupations:
        fig.add_annotation(
            text="Select occupations from the dropdown to view trends",
            xref="paper", yref="paper",
            x=0.5, y=0.5,
            showarrow=False,
            font=dict(size=16)
        )
        fig.update_layout(height=600)
        return fig, warning_message

    # Add lines for each occupation
    colors = px.colors.qualitative.Plotly  # Use Plotly's color scale

    for i, occupation in enumerate(selected_occupations):
        occ_data = trend_df[trend_df['Occupation'] == occupation]
        if not occ_data.empty:
            # Get a subset of the occupation name for the legend to avoid very long labels
            short_name = textwrap.shorten(occupation, width=40, placeholder="...")

            fig.add_trace(go.Scatter(
                x=occ_data['Year'],
                y=occ_data['Value'],
                mode='lines+markers',
                name=short_name,
                line=dict(color=colors[i % len(colors)], width=3),
                marker=dict(size=10)
            ))

    # Update layout
    fig.update_layout(
        height=600,
        xaxis=dict(
            title='Year',
            tickvals=[2010, 2015, 2021],
            dtick=5,
            gridcolor='lightgray'
        ),
        yaxis=dict(
            title='Number of People',
            tickformat=',d',  # Format numbers with commas
            gridcolor='lightgray'
        ),
        hovermode='closest',
        legend=dict(
            orientation='v',
            yanchor='top',
            y=0.99,
            xanchor='right',
            x=0.99,
            bgcolor='rgba(255, 255, 255, 0.8)'
        ),
        margin=dict(l=50, r=50, t=50, b=50)
    )

    return fig, warning_message


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')

<IPython.core.display.Javascript object>